
## Scenarios for sharing models, environments and components

There are two scenarios where you'd want to use the same set of models, components and environments in multiple workspaces. First is cross-workspace MLOps. You are trining a model in a dev workspace and need to deploy it to test and prod workspaces. In this case you want to have end-to-end lineage between endpoints to which the model is deployed in test or prod workspaces and the training job, metrics, code, data and environment that was used to train the model in the dev workspace. Second is to share and reuse models and pipelines across different teams in your organization that in turn improves collaboration and productivity. In this scenario, you may want to publish a trained model and the associated components and environments used to train the model to a central catalog where colleagues from other teams and search and reuse assets shared by you in their experiments. You will learn how to create models, components and environments in a Azure Machine Learning registry and use them in any workspace within your organization.

In [ ]:
pre-req, goals - todo

In [2]:
# Import required libraries
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

from azure.ai.ml import MLClient, Input, Output
from azure.ai.ml.dsl import pipeline
from azure.ai.ml import load_component
from azure.ai.ml.entities import Environment, BuildContext

!pip show azure-ai-ml

Name: azure-ai-ml
Version: 0.1.0b6
Summary: Microsoft Azure Machine Learning Client Library for Python
Home-page: https://github.com/Azure/azure-sdk-for-python
Author: Microsoft Corporation
Author-email: azuresdkengsysadmins@microsoft.com
License: MIT License
Location: /root/miniconda3/lib/python3.8/site-packages
Requires: azure-storage-file-datalake, pathspec, pyjwt, isodate, azure-common, pyyaml, strictyaml, pydash, applicationinsights, msrest, tqdm, marshmallow, azure-mgmt-core, azure-storage-file-share, docker, typing-extensions, azure-core, colorama, azure-identity, azure-storage-blob, jsonschema
Required-by: 


In [4]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

## Connect to a workspace and registry

If you do not have a workspace, refer to this notebook to create a workspace: [](../workspace/workspace.ipynb)
If you do not have a registry, refer to this notebook to create a registry: todo

We will initialize two clients, one to connect to workspace and other connect to registry

In [5]:
ml_client_workspace = MLClient( credential=credential,
    subscription_id = "21d8f407-c4c4-452e-87a4-e609bfb86248",
    resource_group_name = "rg-contoso-819prod",
    workspace_name = "mlw-contoso-819prod")
print(ml_client_workspace)

ml_client_registry = MLClient ( credential=credential,
        subscription_id = "ea4faa5b-5e44-4236-91f6-5483d5b17d14",
        resource_group_name = "bug-bash-rg1",
        registry_name = "ContosoMLjun14")
print(ml_client_registry)

MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x7ff2b7574ee0>,
         subscription_id=21d8f407-c4c4-452e-87a4-e609bfb86248,
         resource_group_name=rg-contoso-819prod,
         workspace_name=mlw-contoso-819prod)
MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x7ff2b7574ee0>,
         subscription_id=ea4faa5b-5e44-4236-91f6-5483d5b17d14,
         resource_group_name=bug-bash-rg1,
         workspace_name=None)


### Create environment in registry

In [13]:
env_docker_context = Environment(
    build=BuildContext(path="../../../cli/jobs/pipelines-with-components/nyc_taxi_data_regression/env_train/"),
    name="SKLearnEnv",
    version=str(1),
    description="Scikit Learn environment",
)
ml_client_registry.environments.create_or_update(env_docker_context)

Uploading env_train (0.0 MBs): 100%|██████████| 520/520 [00:00<00:00, 4517.49it/s]




Environment({'is_anonymous': False, 'auto_increment_version': False, 'name': 'SKLearnEnv', 'description': 'Scikit Learn environment', 'tags': {}, 'properties': {}, 'id': 'azureml://registries/ContosoMLjun14/environments/SKLearnEnv/versions/10', 'Resource__source_path': None, 'base_path': '/mnt/c/CODE/REPOS/azureml-examples/sdk/resources/registry', 'creation_context': <azure.ai.ml._restclient.v2021_10_01_dataplanepreview.models._models_py3.SystemData object at 0x7f1ecc0333d0>, 'serialize': <msrest.serialization.Serializer object at 0x7f1ecc026910>, 'version': '10', 'latest_version': None, 'conda_file': None, 'image': 'mlregcsix4.azurecr.io/sklearnenv_10_c843f0c5-2f5e-57d3-b754-1ecb0f75b360', 'build': None, 'inference_config': None, 'os_type': 'Linux', 'arm_type': 'environment_version', 'conda_file_path': None, 'path': None, 'upload_hash': None, 'translated_conda_file': None})

### Fetch environment from registry

In [21]:
env_from_registry = ml_client_registry.environments.get(name="SKLearnEnv", version=str(1))
print(env_from_registry)

Environment({'is_anonymous': False, 'auto_increment_version': False, 'name': 'SKLearnEnv', 'description': 'Scikit Learn environment', 'tags': {}, 'properties': {}, 'id': 'azureml://registries/ContosoMLjun14/environments/SKLearnEnv/versions/1', 'Resource__source_path': None, 'base_path': '/mnt/c/CODE/REPOS/azureml-examples/sdk/resources/registry', 'creation_context': <azure.ai.ml._restclient.v2021_10_01_dataplanepreview.models._models_py3.SystemData object at 0x7f1ee4743c70>, 'serialize': <msrest.serialization.Serializer object at 0x7f1ecd972160>, 'version': '1', 'latest_version': None, 'conda_file': None, 'image': 'mlregcsix4.azurecr.io/sklearnenv_1_89f98d1d-9796-59d4-8ffb-5985ab0a2237', 'build': None, 'inference_config': None, 'os_type': 'Linux', 'arm_type': 'environment_version', 'conda_file_path': None, 'path': None, 'upload_hash': None, 'translated_conda_file': None})


### Create component in registry

We will use the component YAMLs defined in `cli/jobs/pipelines-with-components/nyc_taxi_data_regression` for this. 

Load the component from YAML

In [12]:
parent_dir = "../../../cli/jobs/pipelines-with-components/nyc_taxi_data_regression"
train_model = load_component(path=parent_dir + "/train.yml")
# print the component as yaml
print(train_model)

$schema: https://azuremlschemas.azureedge.net/latest/commandComponent.schema.json
name: train_linear_regression_model
version: '1'
display_name: TrainLinearRegressionModel
type: command
inputs:
  training_data:
    type: uri_folder
  test_split_ratio:
    type: number
    default: '0.2'
    max: '1.0'
    min: '0.0'
outputs:
  model_output:
    type: mlflow_model
  test_data:
    type: uri_folder
command: python train.py  --training_data ${{inputs.training_data}}  --test_data ${{outputs.test_data}}  --model_output
  ${{outputs.model_output}} --test_split_ratio ${{inputs.test_split_ratio}}
environment: azureml:AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest
code: ./train_src
is_deterministic: true
tags: {}



## 2.2 Create components in registry

Note that we use the `ml_client_registry` handle becuase we plan to create components in registry. Creating the components in Registry allows us to use these components in many workspaces. 

A similar sample notebook shows how to create these components in workspaces instead of registry, in which case you can use those components only in the specific workspace: https://github.com/Azure/azureml-examples/blob/main/sdk/jobs/pipelines/1e_pipeline_with_registered_components/pipeline_with_registered_components.ipynb

We are using a dynamic version number so that there is no conflict with versions that already exist. 

We are also using the environment created in the previous section instead of using the curated environment in the original `train.yml`


In [25]:
# get or create component

# dynamic version number based on epoch time
import time
version_timestamp = str(int(time.time()))
print(version_timestamp)

#train_model.environment="azureml://registries/ContosoMLjun14/environments/SKLearnEnv/versions/1"
train_model.environment=env_from_registry
train_model.version=version_timestamp

#kwargs_version = {"version": version_timestamp }

ml_client_registry.components.create_or_update(train_model)
print(train_model)


1663056411


Creating/updating registry component train_linear_regression_model with version 1663056411 

.

Done (0m 7s)


$schema: https://azuremlschemas.azureedge.net/latest/commandComponent.schema.json
name: train_linear_regression_model
version: '1663056411'
display_name: TrainLinearRegressionModel
type: command
inputs:
  training_data:
    type: uri_folder
  test_split_ratio:
    type: number
    default: '0.2'
    max: '1.0'
    min: '0.0'
outputs:
  model_output:
    type: mlflow_model
  test_data:
    type: uri_folder
command: python train.py  --training_data ${{inputs.training_data}}  --test_data ${{outputs.test_data}}  --model_output
  ${{outputs.model_output}} --test_split_ratio ${{inputs.test_split_ratio}}
environment: azureml://registries/ContosoMLjun14/environments/SKLearnEnv/versions/1
code: azureml://registries/ContosoMLjun14/codes/34af0422-2a89-4b2d-bbf8-bf059bab7de0/versions/1
resources:
  instance_count: 1
  properties: {}
creation_context:
  created_at: '2022-09-13T08:07:09.534401+00:00'
  created_by: Manoj Bableshwar
  created_by_type: User
  last_modified_at: '2022-09-13T08:07:09.5344

### Fetch component from Registry

In [10]:
train_component_from_registry = ml_client_registry.components.get(name="train_linear_regression_model", version=str(1))
print(train_component_from_registry)

$schema: https://azuremlschemas.azureedge.net/latest/commandComponent.schema.json
name: train_linear_regression_model
version: '1'
display_name: TrainLinearRegressionModel
type: command
inputs:
  training_data:
    type: uri_folder
  test_split_ratio:
    type: number
    default: '0.2'
    max: '1.0'
    min: '0.0'
outputs:
  model_output:
    type: mlflow_model
  test_data:
    type: uri_folder
command: python train.py  --training_data ${{inputs.training_data}}  --test_data ${{outputs.test_data}}  --model_output
  ${{outputs.model_output}} --test_split_ratio ${{inputs.test_split_ratio}}
environment: azureml://registries/CuratedRegistry/environments/AzureML-sklearn-0.24-ubuntu18.04-py37-cpu/versions/35
code: azureml://registries/ContosoMLjun14/codes/50142294-866a-4738-b8b3-ce5a42065deb/versions/1
resources:
  instance_count: 1
  properties: {}
tags: {}
is_deterministic: true
creation_context:
  created_at: '2022-06-15T22:20:09.081074+00:00'
  created_by: Manoj Bableshwar
  created_by_

## Create a pipeline job


In [40]:
# Construct pipeline
@pipeline()
def pipeline_with_registered_components(
    training_data
):
    train_job = train_component_from_registry(
        training_data=training_data,
    )
pipeline_job = pipeline_with_registered_components(
    training_data=Input(type="uri_folder", path=parent_dir + "/data_transformed/"),
)
pipeline_job.settings.default_compute = "cpu-cluster"
print(pipeline_job)

display_name: pipeline_with_registered_components
type: pipeline
inputs:
  training_data:
    type: uri_folder
    path: azureml:../../../cli/jobs/pipelines-with-components/nyc_taxi_data_regression/data_transformed/
outputs: {}
jobs:
  train_job:
    $schema: '{}'
    type: command
    inputs:
      training_data:
        path: ${{parent.inputs.training_data}}
    outputs: {}
    command: python train.py  --training_data ${{inputs.training_data}}  --test_data
      ${{outputs.test_data}}  --model_output ${{outputs.model_output}} --test_split_ratio
      ${{inputs.test_split_ratio}}
    code: azureml://registries/ContosoMLjun14/codes/50142294-866a-4738-b8b3-ce5a42065deb/versions/1
    resources:
      instance_count: 1
      properties: {}
    component:
      $schema: https://azuremlschemas.azureedge.net/latest/commandComponent.schema.json
      name: train_linear_regression_model
      version: '1'
      display_name: TrainLinearRegressionModel
      type: command
      inputs:
      

### Submit pipeline job

Submit pipeline job and wait for it to complete


In [41]:
# submit job to workspace
pipeline_job = ml_client_workspace.jobs.create_or_update(
    pipeline_job, experiment_name="sdk_job_component_from_registry" ,  skip_validation=True
)
pipeline_job
ml_client_workspace.jobs.stream(pipeline_job.name)
pipeline_job=ml_client_workspace.jobs.get(pipeline_job.name)
pipeline_job

RunId: happy_angle_9f91rdtctv
Web View: https://ml.azure.com/runs/happy_angle_9f91rdtctv?wsid=/subscriptions/21d8f407-c4c4-452e-87a4-e609bfb86248/resourcegroups/rg-contoso-819prod/workspaces/mlw-contoso-819prod

Streaming logs/azureml/executionlogs.txt

[2022-09-13 19:03:05Z] Completing processing run id c67b1b77-d6ab-4dcb-b111-dfd15185ca43.
[2022-09-13 19:03:05Z] Finishing experiment: no runs left and nothing to schedule.

Execution Summary
RunId: happy_angle_9f91rdtctv
Web View: https://ml.azure.com/runs/happy_angle_9f91rdtctv?wsid=/subscriptions/21d8f407-c4c4-452e-87a4-e609bfb86248/resourcegroups/rg-contoso-819prod/workspaces/mlw-contoso-819prod



Experiment,Name,Type,Status,Details Page
sdk_job_component_from_registry,happy_angle_9f91rdtctv,pipeline,Completed,Link to Azure Machine Learning studio


## Create model in registry

### Option 1: Create model in registry from local files



### Download the model from the output of training job

In [44]:
jobs=ml_client_workspace.jobs.list(parent_job_name=pipeline_job.name)
for job in jobs:
    if (job.display_name == "train_job"):
        print (job.name)
        ml_client_workspace.jobs.download(job.name)

     

c67b1b77-d6ab-4dcb-b111-dfd15185ca43


job c67b1b77-d6ab-4dcb-b111-dfd15185ca43 reuses previous job 8cd07442-fb95-489a-93c0-cc5a41c68c94, download from the reused job.
Ran into a deserialization error. Ignoring since this is failsafe deserialization
Traceback (most recent call last):
  File "/root/miniconda3/lib/python3.8/site-packages/msrest/serialization.py", line 1509, in failsafe_deserialize
    return self(target_obj, data, content_type=content_type)
  File "/root/miniconda3/lib/python3.8/site-packages/msrest/serialization.py", line 1375, in __call__
    data = self._unpack_content(response_data, content_type)
  File "/root/miniconda3/lib/python3.8/site-packages/msrest/serialization.py", line 1543, in _unpack_content
    raise ValueError("This pipeline didn't have the RawDeserializer policy; can't deserialize")
ValueError: This pipeline didn't have the RawDeserializer policy; can't deserialize
Ran into a deserialization error. Ignoring since this is failsafe deserialization
Traceback (most recent call last):
  File "/r

### Create model from local files



In [46]:
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes
import time
version_timestamp = str(int(time.time()))
print(version_timestamp)

mlflow_model = Model(
    path="./artifacts/model/",
    type=AssetTypes.MLFLOW_MODEL,
    name="nyc-taxi-model",
    version=version_timestamp,
    description="MLflow model created from local path",
)
ml_client_registry.create_or_update(mlflow_model)

1663098490


Uploading model (0.0 MBs): 100%|██████████| 2681/2681 [00:00<00:00, 5667.42it/s]




Model({'job_name': None, 'is_anonymous': False, 'auto_increment_version': False, 'name': 'nyc-taxi-model', 'description': 'MLflow model created from local path', 'tags': {}, 'properties': {}, 'id': 'azureml://registries/ContosoMLjun14/models/nyc-taxi-model/versions/1663098490', 'Resource__source_path': None, 'base_path': '/mnt/c/CODE/REPOS/azureml-examples/sdk/resources/registry', 'creation_context': <azure.ai.ml._restclient.v2021_10_01_dataplanepreview.models._models_py3.SystemData object at 0x7ff2abe1ee50>, 'serialize': <msrest.serialization.Serializer object at 0x7ff2b4069880>, 'version': '1663098490', 'latest_version': None, 'path': 'https://mlregcsix4eus1.blob.core.windows.net/contosomlj-c587b14f-a0b7-52e9-98b5-c7d179be76c0/model', 'utc_time_created': None, 'flavors': None, 'arm_type': 'model_version', 'type': 'mlflow_model'})

### Option 2: Copy model from workspace to registry

Get the job name of train_job and build the path pointing to model output 

In [47]:
jobs=ml_client_workspace.jobs.list(parent_job_name=pipeline_job.name)
for job in jobs:
    if (job.display_name == "train_job"):
        print (job.name)
        model_path_from_job="azureml://jobs/{job_name}/outputs/default/model".format(job_name=job.name)

print(model_path_from_job)
     

c67b1b77-d6ab-4dcb-b111-dfd15185ca43
azureml://jobs/c67b1b77-d6ab-4dcb-b111-dfd15185ca43/outputs/default/model


### Create model in workspace from job output

In [49]:
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes
import time
version_timestamp = str(int(time.time()))
print(version_timestamp)

mlflow_model = Model(
    path=model_path_from_job,
    type=AssetTypes.MLFLOW_MODEL,
    name="nyc-taxi-model",
    version=version_timestamp,
    description="MLflow model created from job output",
)
ml_client_workspace.create_or_update(mlflow_model)

1663099093


Model({'job_name': 'c67b1b77-d6ab-4dcb-b111-dfd15185ca43', 'is_anonymous': False, 'auto_increment_version': False, 'name': 'nyc-taxi-model', 'description': 'MLflow model created from job output', 'tags': {}, 'properties': {}, 'id': '/subscriptions/21d8f407-c4c4-452e-87a4-e609bfb86248/resourceGroups/rg-contoso-819prod/providers/Microsoft.MachineLearningServices/workspaces/mlw-contoso-819prod/models/nyc-taxi-model/versions/1663099093', 'Resource__source_path': None, 'base_path': '/mnt/c/CODE/REPOS/azureml-examples/sdk/resources/registry', 'creation_context': <azure.ai.ml._restclient.v2022_05_01.models._models_py3.SystemData object at 0x7ff2b4137a60>, 'serialize': <msrest.serialization.Serializer object at 0x7ff2b536ff10>, 'version': '1663099093', 'latest_version': None, 'path': 'azureml://subscriptions/21d8f407-c4c4-452e-87a4-e609bfb86248/resourceGroups/rg-contoso-819prod/workspaces/mlw-contoso-819prod/datastores/workspaceartifactstore/paths/ExperimentRun/dcid.8cd07442-fb95-489a-93c0-cc5

## Copy a model from workspace to registry

In [54]:

#model_path_from_workspace="azureml://subscriptions/<subscription-id-of-workspace>/resourceGroups/<resource-group-of-workspace>/workspaces/<workspace-name>/models/<model-name>/versions/<model-version>

model_path_from_workspace="azureml://subscriptions/21d8f407-c4c4-452e-87a4-e609bfb86248/resourceGroups/rg-contoso-819prod/workspaces/mlw-contoso-819prod/models/nyc-taxi-model/versions/1663099093"

print(model_path_from_workspace)

mlflow_model = Model(
    path=model_path_from_workspace,
#    type=AssetTypes.MLFLOW_MODEL,
#    name="nyc-taxi-model",
#    version=version_timestamp,
#    description="MLflow model created from job output",
)
ml_client_registry.create_or_update(mlflow_model)
     

azureml://subscriptions/21d8f407-c4c4-452e-87a4-e609bfb86248/resourceGroups/rg-contoso-819prod/workspaces/mlw-contoso-819prod/models/nyc-taxi-model/versions/1663099093


FileNotFoundError: [Errno 2] No such file or directory: 'azureml://subscriptions/21d8f407-c4c4-452e-87a4-e609bfb86248/resourceGroups/rg-contoso-819prod/workspaces/mlw-contoso-819prod/models/nyc-taxi-model/versions/1663099093'

### Deploy model from registry to online endpoint in workspace

### Get model from registry

In [55]:
mlflow_model_from_registry = ml_client_registry.models.get(name="nyc-taxi-model", version=str(1))
print(mlflow_model_from_registry)

Model({'job_name': None, 'is_anonymous': False, 'auto_increment_version': False, 'name': 'nyc-taxi-model', 'description': None, 'tags': {}, 'properties': {}, 'id': 'azureml://registries/ContosoMLjun14/models/nyc-taxi-model/versions/1', 'Resource__source_path': None, 'base_path': '/mnt/c/CODE/REPOS/azureml-examples/sdk/resources/registry', 'creation_context': <azure.ai.ml._restclient.v2021_10_01_dataplanepreview.models._models_py3.SystemData object at 0x7ff2abe1e280>, 'serialize': <msrest.serialization.Serializer object at 0x7ff2b401a9a0>, 'version': '1', 'latest_version': None, 'path': 'https://mlregcsix4eus1.blob.core.windows.net/contosomlj-24ba981e-9e0e-5420-b7ae-271a6eb0367f/model', 'utc_time_created': None, 'flavors': None, 'arm_type': 'model_version', 'type': 'custom_model'})


### Create an online endpoint 

In [57]:
import datetime

from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
)

online_endpoint_name = "endpoint-" + datetime.datetime.now().strftime("%m%d%H%M%f")

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="this is a sample online endpoint for mlflow model",
    auth_mode="key"
)
ml_client_workspace.begin_create_or_update(endpoint)

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://endpoint-09131433696352.eastus.inference.ml.azure.com/score', 'swagger_uri': 'https://endpoint-09131433696352.eastus.inference.ml.azure.com/swagger.json', 'name': 'endpoint-09131433696352', 'description': 'this is a sample online endpoint for mlflow model', 'tags': {}, 'properties': {'azureml.onlineendpointid': '/subscriptions/21d8f407-c4c4-452e-87a4-e609bfb86248/resourcegroups/rg-contoso-819prod/providers/microsoft.machinelearningservices/workspaces/mlw-contoso-819prod/onlineendpoints/endpoint-09131433696352', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/21d8f407-c4c4-452e-87a4-e609bfb86248/providers/Microsoft.MachineLearningServices/locations/eastus/mfeOperationsStatus/oe:4b0187f3-cadc-4d54-91fe-55b6c38c8b45:3d145cb2-dcb8-4bfc-a567-b1cec0bb8871?api-version=2022-02-01-preview'}, 'id': '/subscriptions/21d8f407-c4c4-452e-87a4-e609bfb86248/resourc

### Deploy the model from registry to the online endpoint

In [59]:
# create a blue deployment
demo_deployment = ManagedOnlineDeployment(
    name="demo",
    endpoint_name=online_endpoint_name,
    model=mlflow_model_from_registry,
    instance_type="Standard_F4s_v2",
    instance_count=1
)
ml_client_workspace.online_deployments.begin_create_or_update(demo_deployment)


Check: endpoint endpoint-09131433696352 exists


ModelException: Error with model: (UserError) Request is invalid and/or missing fields.
Code: UserError
Message: Request is invalid and/or missing fields.
Exception Details:	(RequestInvalid) Error: Property: ModelUri does not match the regex: ^(azureml://subscriptions/([^/?]+)/resourceGroups/([^/?]+)/workspaces/([^/?]+)/datastores/([^/?]+)/paths/(.+)|azureml://jobs/([^/]+)/outputs/([^/]+)(.*)|runs:/([^/?]+)/(.+))$
	Code: RequestInvalid
	Message: Error: Property: ModelUri does not match the regex: ^(azureml://subscriptions/([^/?]+)/resourceGroups/([^/?]+)/workspaces/([^/?]+)/datastores/([^/?]+)/paths/(.+)|azureml://jobs/([^/]+)/outputs/([^/]+)(.*)|runs:/([^/?]+)/(.+))$
Additional Information:Type: ComponentName
Info: {
    "value": "managementfrontend"
}Type: Correlation
Info: {
    "value": {
        "operation": "5b04f4e67deb4d8b2c2ec9c9163168d3",
        "request": "450893fe08ca7b93"
    }
}Type: Environment
Info: {
    "value": "eastus"
}Type: Location
Info: {
    "value": "eastus"
